In [19]:
!python -m spacy download pt_core_news_lg
!pip install emoji
!pip install --upgrade googletrans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 779.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [100]:
import spacy
import pandas as pd
import unicodedata
import re
import emoji

In [101]:
nlp_pt = spacy.load("pt_core_news_lg")

In [102]:
data_file = 'textos.csv'

In [103]:
df = pd.read_csv(data_file, usecols=['text']).sample(n=10, random_state=23, ignore_index=True).drop_duplicates(subset=['text'])
df.head(7)

,text
0,A derrubada do veto do presidente @LulaOficial...
2,Y ahora que dirá Doña @MaElviraSalazar ?\nEl c...
4,"Estive em Estrela, ao lado do ministro @waldez..."
5,@summarizest @LulaOficial ...qual droga usou s...
7,"☠️⚰️ "" O EX-PRESIDIÁRIO e LADRÃO Luís Inácio L..."
9,"Já são mais de 30 dias vendo cenas tristes, de..."


In [184]:
#PREGUNTA 1: EMOJIS/ Enlistamos emojis y vemos cuantas veces se repiten en los tweets


# Cargar el DataFrame desde el archivo CSV
df = pd.read_csv('textos.csv')

# Seleccionar la columna de texto en la que deseas detectar emojis
columna_texto = 'name'

# Función para contar emojis en un texto
def contar_emojis(texto):
    return len([char for char in texto if emoji.is_emoji(char)])

# Aplicar la función contar_emojis a la columna de texto seleccionada
df['emoji_count'] = df[columna_texto].apply(contar_emojis)

# Crear un diccionario para almacenar la frecuencia de cada emoji
emoji_frecuencia = {}
for index, row in df.iterrows():
    emojis_texto = [char for char in row[columna_texto] if emoji.is_emoji(char)]
    for emoji_char in emojis_texto:
        emoji_name = emoji.demojize(emoji_char)
        if emoji_name in emoji_frecuencia:
            emoji_frecuencia[emoji_name] += 1
        else:
            emoji_frecuencia[emoji_name] = 1

# Obtener todos los emojis únicos
emojis_unicos = set(emoji_frecuencia.keys())

print("Total de emojis encontrados:", sum(df['emoji_count']))
print("Emojis únicos", emojis_unicos)


# Crear DataFrame para mostrar los resultados
Tabla = pd.DataFrame({'Emoji': list(emoji_frecuencia.keys()), 'Frecuencia': list(emoji_frecuencia.values())})

# Imprimir los primeros 20 registros del DataFrame
print(Tabla.head(20))




Total de emojis encontrados: 103
Emojis únicos {':yellow_circle:', ':rainbow:', ':water_wave:', ':female_sign:', ':peace_symbol:', ':white_heart:', ':light_skin_tone:', ':cactus:', ':soccer_ball:', ':airplane:', ':victory_hand:', ':bright_button:', ':paw_prints:', ':spouting_whale:', ':revolving_hearts:', ':black_flag:', ':watermelon:', ':medium_skin_tone:', ':doughnut:', ':registered:', ':globe_showing_Americas:', ':face_with_medical_mask:', ':syringe:', ':triangular_flag:', ':maple_leaf:', ':white_flag:', ':heart_suit:', ':green_heart:', ':black_heart:', ':purple_circle:', ':blue_heart:', ':eagle:', ':handshake:', ':red_circle:', ':reminder_ribbon:', ':smiling_face_with_sunglasses:', ':FREE_button:', ':black_circle:', ':droplet:', ':yellow_heart:', ':red_triangle_pointed_down:', ':dove:', ':radioactive:', ':squid:', ':crocodile:', ':play_button:'}
                             Emoji  Frecuencia
0                :triangular_flag:          39
1   :smiling_face_with_sunglasses:          

In [186]:
#PREGUNTA 2: LIMPIEZA DE TEXTO


from spacy.lang.pt.stop_words import STOP_WORDS


# Función para limpiar el texto
def limpiar_texto(texto):
    # Eliminar caracteres especiales y puntuación
    texto_limpio = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚ\s]', '', texto)
    # Convertir texto a minúsculas
    texto_limpio = texto_limpio.lower()
    # Tokenizar el texto
    tokens = nlp_pt(texto_limpio)
    # Eliminar palabras vacías (stopwords)
    tokens_limpio = [token.lemma_ for token in tokens if not token.is_stop]
    # Unir las palabras nuevamente en un texto
    texto_limpio = ' '.join(tokens_limpio)
    return texto_limpio

# Leer el archivo CSV y seleccionar solo la columna 'text'
data_file = 'textos.csv'
df = pd.read_csv(data_file, usecols=['text'])

# Eliminar filas con valores nulos
df = df.dropna()

# Aplicar la función de limpieza al DataFrame
df['text_clean'] = df['text'].apply(limpiar_texto)


print(df.head(10))

                                                text  \
0  O presidente Luiz Inácio Lula da Silva (PT) en...   
1  A derrubada do veto do presidente @LulaOficial...   
2  A derrubada do veto do presidente @LulaOficial...   
3  Analfabeto,esperto,sem capacidade,metido a dit...   
4  A derrubada do veto do presidente @LulaOficial...   
5  @roberta_bastoss Responda uma coisinha para a ...   
6  A derrubada do veto do presidente @LulaOficial...   
7  @LulaOficial Fala meu presidente que não pode ...   
8  @brasil247 Grandes ou pífios bajuladores do ex...   
9  @gleisi @LulaOficial André Mendonça salvando o...   

                                          text_clean  
0  presidente Luiz inácio Lula Silva pt encaminha...  
1  derrubada veto presidente lulaoficial saidinha...  
2  derrubada veto presidente lulaoficial saidinha...  
3  analfabetoespertosem capacidademetir ditadorlu...  
4  derrubada veto presidente lulaoficial saidinha...  
5  robertabastoss responda coisinha galera \n\n v... 

In [187]:
#Anotaciones Con Spacy
for i, doc in enumerate([nlp_pt(text) for text in df['text_clean'].values[:5]]):
  print(f'\n{i + 1}. {doc.text}\n')
  for token in doc:
    text_doc = ''
    text_doc += f'[TOKEN] -> {token.text} '
    if token.lemma_ != token.text:
      text_doc += f'[Lemma] -> {token.lemma_} '
    text_doc += f'[POS] -> {token.pos_} [TAG] -> {token.tag_} [DEP] -> {token.dep_} [SHAPE] -> {token.shape_} [ALPHA] -> {token.is_alpha} [STOP] -> {token.is_stop}'

    print(text_doc)


1. presidente Luiz inácio Lula Silva pt encaminhar apóstolo ester hernandez criador marcha Jesus realizar quintafeira   so Paulo carta dizer honra dimenso capacidade evento reunir milhar fiel diferentes igreja Crists Brasil mundo 

 Cristo sintome regozijar dimenso extraordinário evento tomar papel significativo desempenhar vida brasileiro promover valor paz fé amor solidariedade dizer Lula 

 acompanhar desdobramento aes governo federal rio sul presidente representar marcha ministro advocaciageral unio agu Jorge messia 

  reuterscarla carniel 
  informaes site bandcombr 

 bandjornalismo

[TOKEN] -> presidente [POS] -> NOUN [TAG] -> NOUN [DEP] -> nsubj [SHAPE] -> xxxx [ALPHA] -> True [STOP] -> False
[TOKEN] -> Luiz [POS] -> PROPN [TAG] -> PROPN [DEP] -> appos [SHAPE] -> Xxxx [ALPHA] -> True [STOP] -> False
[TOKEN] -> inácio [POS] -> PROPN [TAG] -> PROPN [DEP] -> flat:name [SHAPE] -> xxxx [ALPHA] -> True [STOP] -> False
[TOKEN] -> Lula [POS] -> PROPN [TAG] -> PROPN [DEP] -> flat:name

In [188]:
#PREGUNTA 3: ADJETIVOS

adj = []
for i, doc in enumerate( [nlp_pt(text) for text in df['text_clean'].values [:5]]):
#print(f'\n{i + 1}. {doc. text}\n')
 for token in doc:
   text_doc = ''
   text_doc= f' [TOKEN] -> {token.text} '
#Comprobamos si es igual, a la forma base del arbo casa contrario lo colocamos
   if token.lemma_ != token.text:
      text_doc += f' [Lemma] -> {token.lemma_}'
      text_doc += f' [POS] → {token.pos_} [TAG] → {token.tag_} [DEP] → {token.dep_}'
  # print(f'Token: {token.text}, POS: {token.pos_}')
   if token.pos_ == "ADJ":
    #token. lemma_ es la forma base del verbo
    adj.append(token.lemma_)

#print(text_doc)
Tabla = pd.DataFrame( {'Adj': adj})
Tabla.head (20)

,Adj
0,ester
1,dimenso
2,evento
3,fiel
4,sintome
5,dimenso
6,extraordinário
7,significativo
8,brasileiro
9,aes


In [189]:
#PREGUNTA 3.1: CONTAR LA FRECUENCIA DE LOS ADJETIVOS



from collections import Counter

# Contar la frecuencia de adjetivos
conteo_adjetivos = Counter(adj)

# Crear un DataFrame para almacenar la frecuencia de adjetivos
adjetivos_df = pd.DataFrame(conteo_adjetivos.items(), columns=['Adjetivo', 'Frecuencia'])

# Ordenar el DataFrame por frecuencia de adjetivos en orden descendente
adjetivos_df = adjetivos_df.sort_values(by='Frecuencia', ascending=False)

# Reiniciar el índice del DataFrame
adjetivos_df = adjetivos_df.reset_index(drop=True)

print(adjetivos_df.head(10))


          Adjetivo  Frecuencia
0      lulaoficial           3
1          cumprer           3
2       temporário           3
3         restrior           3
4           deciso           3
5          dimenso           2
6    ortegamaduroe           1
7           latino           1
8  pífiosditadores           1
9            chefe           1


In [190]:
#Cargamos nuestro diccionario de sentimientos traducido a portuges
sentimientos = 'dicsent.txt'
sent = pd.read_csv(sentimientos, delimiter=',', encoding='utf-8', names=['word', 'sent'])
sent.head(15)

,word,sent
0,abandonar,-2
1,abandonado,-2
2,abandonar,-2
3,sequestrado,-2
4,raptos,-2
5,abominar,-3
6,abominável,-3
7,abomina,-3
8,habilidades,2
9,habilidade,2


In [191]:
df['words'] = df['text_clean'].apply(lambda x: x.split())
df['num_words'] = df['words'].apply(len)

# Mostrar el resultado
print(df[['text_clean', 'words', 'num_words']].head())


                                          text_clean  \
0  presidente Luiz inácio Lula Silva pt encaminha...   
1  derrubada veto presidente lulaoficial saidinha...   
2  derrubada veto presidente lulaoficial saidinha...   
3  analfabetoespertosem capacidademetir ditadorlu...   
4  derrubada veto presidente lulaoficial saidinha...   

                                               words  num_words  
0  [presidente, Luiz, inácio, Lula, Silva, pt, en...         73  
1  [derrubada, veto, presidente, lulaoficial, sai...         26  
2  [derrubada, veto, presidente, lulaoficial, sai...         26  
3  [analfabetoespertosem, capacidademetir, ditado...         20  
4  [derrubada, veto, presidente, lulaoficial, sai...         26  


In [192]:
def asignarSent(texto):
    words = texto.split()
    wordsSentTxt = []
    if len(words) > 0:
        for w in words:
            val = sent.loc[sent['word'] == w]
            if not val.empty:
                wordsSentTxt.append(val['sent'].iloc[0])
    return wordsSentTxt

In [193]:
df['sentPalabras'] = df['text_clean'].apply(asignarSent)
print(df.head(15))


                                                 text  \
0   O presidente Luiz Inácio Lula da Silva (PT) en...   
1   A derrubada do veto do presidente @LulaOficial...   
2   A derrubada do veto do presidente @LulaOficial...   
3   Analfabeto,esperto,sem capacidade,metido a dit...   
4   A derrubada do veto do presidente @LulaOficial...   
5   @roberta_bastoss Responda uma coisinha para a ...   
6   A derrubada do veto do presidente @LulaOficial...   
7   @LulaOficial Fala meu presidente que não pode ...   
8   @brasil247 Grandes ou pífios bajuladores do ex...   
9   @gleisi @LulaOficial André Mendonça salvando o...   
10  Lula não vai à Marcha para Jesus em São Paulo ...   
11  Eu prometo que faremos de tudo para devolver b...   
12  Eu prometo que faremos de tudo para devolver b...   
13  @gleisi @LulaOficial Na verdade, senhora, não ...   
14  Eu prometo que faremos de tudo para devolver b...   

                                           text_clean  \
0   presidente Luiz inácio Lul

In [195]:
def asignarSent(texto):
    words = texto.split()
    wordsSentTxt = []
    if len(words) > 0:
        for w in words:
            val = sent.loc[sent['word'] == w]
            if not val.empty:
                wordsSentTxt.append(val['sent'].iloc[0])
df['sentPalabras'] = df['text_clean'].apply(asignarSent)



In [197]:
suma_tmp = 0
# Iterar sobre los elementos de tmp
for sent in tmp:
    try:
        # Intentar convertir el elemento a un entero y agregarlo a la suma
        suma_tmp += int(sent)
    except ValueError:
        # Si no se puede convertir a un entero, imprimir un mensaje de error
        print(f"El elemento '{sent}' no es un número entero")

# Imprimir la muestra y su suma correspondiente
print(tmp)
print(f"Suma:  {suma_tmp}")


['3', '-2']
Suma:  1
